# Notebook Instructions

You can run the notebook document sequentially (one cell at a time) by pressing shift + enter. While a cell is running, a [*] will display on the left. When it has been run, a number will display indicating the order in which it was run in the notebook [8].

Enter edit mode by pressing Enter or using the mouse to click on a cell's editor area. Edit mode is indicated by a green cell border and a prompt showing in the editor area.

## Class Weights in Decision Trees

When we are building a decision tree model, it can happen that the dataset provided to the model may have very less data points for its most important classes. In such an instance, the decision tree algorithm will try to maximize the accuracy of the most common labels. 

In order to adjust for this issue, we re-assign weights to the data points of the most important labels. This can be done in the scikit-library using the class_weight argument to the decision tree classifier. Let us take an example to illustrate this.

#### Example:

We will input raw data of ACC Ltd. stock from a csv file. The data consists of Open-High-Low-Close prices and Volume data. Predictor and target variables are created using this raw data. 

In [80]:
import pandas as pd
df = pd.read_csv("ACC.csv")

df.tail()

,DATE,OPEN,HIGH,LOW,CLOSE,VOLUME
243,6/11/2018,1315.5,1338.80,1312.05,1320.20,91264
244,6/12/2018,1325.0,1329.00,1306.55,1315.70,81244
245,6/13/2018,1320.2,1349.40,1313.05,1330.95,144451
246,6/14/2018,1331.5,1332.55,1304.45,1307.20,72010
247,6/15/2018,1308.5,1314.80,1295.00,1302.10,142994


#### Computing Technical Indicators and Daily Future Returns

We compute the values for the Average Directional Index (ADI), Relative Strength Index (RSI), and Simple Moving Average (SMA) using the TA-Lib package. These will be used as predictor variables in the decision tree model. Next, we compute the daily future returns on the close price. The code is shown below.


In [81]:
import numpy as np
import talib as ta 

df['ADX'] = ta.ADX(df['HIGH'].values, df['LOW'].values, df['CLOSE'].values, timeperiod=14)
df['RSI'] = ta.RSI(df['CLOSE'].values, timeperiod=14)
df['SMA'] = ta.SMA(df['CLOSE'].values, timeperiod=20)

df['Return'] = df['CLOSE'].pct_change(1).shift(-1)  
df = df.dropna()

df.tail()

,DATE,OPEN,HIGH,LOW,CLOSE,VOLUME,ADX,RSI,SMA,Return
242,6/8/2018,1314.0,1325.55,1305.50,1315.10,100396,42.429498,34.919144,1342.5550,0.003878
243,6/11/2018,1315.5,1338.80,1312.05,1320.20,91264,40.894421,36.473826,1336.0050,-0.003409
244,6/12/2018,1325.0,1329.00,1306.55,1315.70,81244,39.620536,35.664267,1331.2075,0.011591
245,6/13/2018,1320.2,1349.40,1313.05,1330.95,144451,37.586661,40.485233,1327.2150,-0.017844
246,6/14/2018,1331.5,1332.55,1304.45,1307.20,72010,35.965272,35.965189,1322.5500,-0.003901


#### Categorize Returns into Multiple Classes

We define a function called 'returns_to_class' using nested If..else statement to categorize returns into multiple classes. We also specify the range for the returns for each class in this function. This function is then applied on our dataframe, df to get the multi-class target variable.


In [82]:
def returns_to_class(df):
    if df.Return <= 0.0:
        return 0
    elif df.Return > 0.0 and df.Return < 0.02:
        return 1
    elif df.Return > 0.02 and df.Return< 0.03:
        return 2
    else:
        return 3

df['Class'] = df.apply(returns_to_class,axis=1)
df.tail()

,DATE,OPEN,HIGH,LOW,CLOSE,VOLUME,ADX,RSI,SMA,Return,Class
242,6/8/2018,1314.0,1325.55,1305.50,1315.10,100396,42.429498,34.919144,1342.5550,0.003878,1
243,6/11/2018,1315.5,1338.80,1312.05,1320.20,91264,40.894421,36.473826,1336.0050,-0.003409,0
244,6/12/2018,1325.0,1329.00,1306.55,1315.70,81244,39.620536,35.664267,1331.2075,0.011591,1
245,6/13/2018,1320.2,1349.40,1313.05,1330.95,144451,37.586661,40.485233,1327.2150,-0.017844,0
246,6/14/2018,1331.5,1332.55,1304.45,1307.20,72010,35.965272,35.965189,1322.5500,-0.003901,0


#### View the Multi-Class Distribution

Once we have defined the different classes for the target variable, we can see their distribution of Returns using the groupby method. As can be observed, out of the total data points majority of them (i.e. 126 data points) belong to '0' class which signifies negative returns. On the other hand, there are only 11 and 1 datapoint belonging to the '2' and the '3' class respectively.

In [83]:
df.groupby('Class').count()

,DATE,OPEN,HIGH,LOW,CLOSE,VOLUME,ADX,RSI,SMA,Return
Class,,,,,,,,,,
0,126,126,126,126,126,126,126,126,126,126
1,82,82,82,82,82,82,82,82,82,82
2,9,9,9,9,9,9,9,9,9,9
3,3,3,3,3,3,3,3,3,3,3


#### Create Predictor Variables and Target Variable

Let us now define our predictors variables, X and the target variable, y for building a decision tree model.

In [84]:
X = df[['ADX','RSI','SMA']]
y = df.Class

  
We will consider two scenarios:   

1) Building a decision tree model without applying the class weights and    
2) Building a decision tree model with class weights.


### Scenario 1 - Build a decision tree model without applying the Class weights 

In [85]:
# Split into Train and Test datasets 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=42,stratify=y)
  
#print (X_train.shape, y_train.shape)
#print (X_test.shape, y_test.shape)

# Fit a model on train data
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(criterion='gini', max_depth=3, min_samples_leaf=5) 
clf = clf.fit(X_train, y_train)

# Use the trained model to make predictions on the test data
y_pred = clf.predict(X_test)  
  
# Evaluate the model performance 
from sklearn.metrics import classification_report
report = classification_report(y_test, y_pred)
print (report)

             precision    recall  f1-score   support

          0       0.55      0.84      0.67        38
          1       0.12      0.04      0.06        24
          2       0.00      0.00      0.00         3
          3       0.00      0.00      0.00         1

avg / total       0.36      0.50      0.41        66



C:\Users\academy\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


As can be seen from the output of the classification report, the decision tree algorithm tries to maximize the accuracy of the most common labels and does not give good predictions on the underrepresented labels.

### Scenario 2 - Build a decision tree model with Class weights 

Let us use the class_weight parameter when defining the decision tree classifier to correct for the underrepresented labels.

We can assigned class_weight = 'balanced'. This re-weighting of the data points causes the classes to appear with equal frequency.

As can be seen from the output of the classification report, using of class weight makes the decision tree algorithm achieve higher accuracy on the underrepresented labels which were labels '2'and '3' in this case.

In [86]:
# Split into Train and Test datasets 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=42,stratify=y)


# Fit a model on train data
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(criterion = 'gini', max_depth = 3, min_samples_leaf = 5,
                            class_weight = 'balanced') 
clf = clf.fit(X_train, y_train)

# Use the trained model to make predictions on the test data
y_pred = clf.predict(X_test)  
  
# Evaluate the model performance 
from sklearn.metrics import classification_report
report = classification_report(y_test, y_pred)
print (report)

             precision    recall  f1-score   support

          0       0.64      0.24      0.35        38
          1       0.29      0.08      0.13        24
          2       0.03      0.33      0.05         3
          3       0.20      1.00      0.33         1

avg / total       0.48      0.20      0.25        66

